### Notebook to determine which point of data_traite from dataAnalysis presents the biggest permeability value

In [18]:
#Necessary imports

from pyheatmy import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import chardet
import csv
import os

In [19]:
path = 'C:/Users/alexa/2A/MOLONARI1D/dataAnalysis/data_traite'

print("Les points de mesures, en température et en pression, sont les suivants :")

os.listdir(path)[1:]

Les points de mesures, en température et en pression, sont les suivants :


['point13_pression_traité.csv',
 'point13_temperature_traité.csv',
 'point14_pression_traité.csv',
 'point14_temperature_traité.csv',
 'point34_pression_traité.csv',
 'point34_temperature_traité.csv',
 'point35_pression_traité.csv',
 'point35_temperature_traité.csv',
 'point36_pression_traité.csv',
 'point36_temperature_traité.csv',
 'point46_pression_traité.csv',
 'point46_temperature_traité.csv',
 'point47_pression_traité.csv',
 'point47_temperature_traité.csv',
 'point48_pression_traité.csv',
 'point48_temperature_traité.csv',
 'point52_pression_traité.csv',
 'point52_temperature_traité.csv',
 'point53_pression_traité.csv',
 'point53_temperature_traité.csv']

In [20]:
#Fonction pour convertir les dates en datetime

def convertDates(df: pd.DataFrame):
    """
    Convert dates from a list of strings by testing several different input formats
    Try all date formats already encountered in data points
    If none of them is OK, try the generic way (None)
    If the generic way doesn't work, this method fails
    (in that case, you should add the new format to the list)
    
    This function works directly on the giving Pandas dataframe (in place)
    This function assumes that the first column of the given Pandas dataframe
    contains the dates as characters string type
    
    For datetime conversion performance, see:
    See https://stackoverflow.com/questions/40881876/python-pandas-convert-datetime-to-timestamp-effectively-through-dt-accessor
    """
    formats = ("%m/%d/%y %H:%M:%S", "%m/%d/%y %I:%M:%S %p",
               "%d/%m/%y %H:%M",    "%d/%m/%y %I:%M %p",
               "%m/%d/%Y %H:%M:%S", "%m/%d/%Y %I:%M:%S %p", 
               "%d/%m/%Y %H:%M",    "%d/%m/%Y %I:%M %p",
               "%y/%m/%d %H:%M:%S", "%y/%m/%d %I:%M:%S %p", 
               "%y/%m/%d %H:%M",    "%y/%m/%d %I:%M %p",
               "%Y/%m/%d %H:%M:%S", "%Y/%m/%d %I:%M:%S %p", 
               "%Y/%m/%d %H:%M",    "%Y/%m/%d %I:%M %p",
               None)
    times = df[df.columns[0]]
    for f in formats:
        try:
            # Convert strings to datetime objects
            new_times = pd.to_datetime(times, format=f)
            # Convert datetime series to numpy array of integers (timestamps)
            new_ts = new_times.values.astype(np.int64)
            # If times are not ordered, this is not the appropriate format
            test = np.sort(new_ts) - new_ts
            if np.sum(abs(test)) != 0 :
                #print("Order is not the same")
                raise ValueError()
            # Else, the conversion is a success
            #print("Found format ", f)
            df[df.columns[0]] = new_times
            return
        
        except ValueError:
            #print("Format ", f, " not valid")
            continue
    
    # None of the known format are valid
    raise ValueError("Cannot convert dates: No known formats match your data!")

#Fonction pour lire les fichiers csv

#Lecture des données

def read_csv (chemin_fichier):
    #Detecter separateur
    with open(chemin_fichier, 'r') as file:
        sniffer = csv.Sniffer()
        sample_data = file.read(1024)
        detecter_separateur = (sniffer.sniff(sample_data).delimiter)

    if "Titre" in open(chemin_fichier).readline():
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur, skiprows=1)
    else:
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur)

    return data_frame 

In [21]:
#Point considéré

point_pression = 'point13_pression_traité.csv'
point_temperature = 'point13_temperature_traité.csv'

#Définition d'un dictionnaire pour les données 

dico = {}
dico['chemin pression'] = os.path.join(path, point_pression)
dico['chemin temperature'] = os.path.join(path, point_temperature)

pression = read_csv(dico['chemin pression'])
temperature = read_csv(dico['chemin temperature'])

pression['temperature_stream'] = pression['temperature_stream'] + ZERO_CELSIUS
temperature['Temp_profondeur_10'] = temperature['Temp_profondeur_10'] + ZERO_CELSIUS
temperature['Temp_profondeur_20'] = temperature['Temp_profondeur_20'] + ZERO_CELSIUS
temperature['Temp_profondeur_30'] = temperature['Temp_profondeur_30'] + ZERO_CELSIUS
temperature['Temp_profondeur_40'] = temperature['Temp_profondeur_40'] + ZERO_CELSIUS

#Conversion des dates en datetime   
convertDates(pression)
convertDates(temperature)

dico['pression'] = pression
dico['temperature'] = temperature

print(dico['pression'].head())
print(dico['temperature'].head())

dico['temperature'].columns = ['dates', 'T_10', 'T_20', 'T_30', 'T_40']

                dates  temperature_stream        dH
0 2015-07-23 14:00:00             291.374  0.085864
1 2015-07-23 14:15:00             291.231  0.086153
2 2015-07-23 14:30:00             291.207  0.086189
3 2015-07-23 14:45:00             291.207  0.086257
4 2015-07-23 15:00:00             291.326  0.086072
                dates  Temp_profondeur_10  Temp_profondeur_20  \
0 2015-07-23 14:00:00             291.469             291.088   
1 2015-07-23 14:15:00             291.374             290.803   
2 2015-07-23 14:30:00             291.255             290.708   
3 2015-07-23 14:45:00             291.231             290.684   
4 2015-07-23 15:00:00             291.207             290.660   

   Temp_profondeur_30  Temp_profondeur_40  
0             291.397             291.540  
1             291.065             291.136  
2             290.946             290.946  
3             290.898             290.898  
4             290.874             290.874  


C:\Users\alexa\AppData\Local\Temp\ipykernel_15056\2964212467.py:31: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  new_times = pd.to_datetime(times, format=f)
C:\Users\alexa\AppData\Local\Temp\ipykernel_15056\2964212467.py:31: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  new_times = pd.to_datetime(times, format=f)


In [22]:
#On va maintenant compute la MCMC pour les données de pression et de température, pour trouver une valeur de 
#perméabilité pour chaque point de mesure

#On commence par créer des listes de tuple pour les données de pression et de température

data_pression = list(zip(dico['pression']['dates'], list(zip( dico['pression']['dH'], dico['pression']['temperature_stream']))))
data_temperature = list(zip(dico['temperature']['dates'], dico['temperature'][['T_10', 'T_20', 'T_30', 'T_40']].to_numpy()))

print (data_pression[:5])
print (data_temperature[:5])

[(Timestamp('2015-07-23 14:00:00'), (0.0858643254074198, 291.37399999999997)), (Timestamp('2015-07-23 14:15:00'), (0.0861528697161693, 291.231)), (Timestamp('2015-07-23 14:30:00'), (0.0861893984470073, 291.207)), (Timestamp('2015-07-23 14:45:00'), (0.0862567747756304, 291.207)), (Timestamp('2015-07-23 15:00:00'), (0.0860715126342083, 291.32599999999996))]
[(Timestamp('2015-07-23 14:00:00'), array([291.469, 291.088, 291.397, 291.54 ])), (Timestamp('2015-07-23 14:15:00'), array([291.374, 290.803, 291.065, 291.136])), (Timestamp('2015-07-23 14:30:00'), array([291.255, 290.708, 290.946, 290.946])), (Timestamp('2015-07-23 14:45:00'), array([291.231, 290.684, 290.898, 290.898])), (Timestamp('2015-07-23 15:00:00'), array([291.207, 290.66 , 290.874, 290.874]))]


In [37]:
#On initialise les paramètres de la MCMC et de l'objet column à partir d'un dico

Z_bottom = .4
depth_sensors = [0.1, 0.2, 0.3, Z_bottom]

col_dict = {
	"river_bed": .4, 
    "depth_sensors": depth_sensors, #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"offset": .0,
    "dH_measures": data_pression,
    "T_measures": data_temperature,
    "sigma_meas_P": 0.01, #float
    "sigma_meas_T": 0.1, #float
}

col = Column.from_dict(col_dict)

priors = {
    "moinslog10IntrinK": ((10, 16), .01), # (intervalle, sigma)
    "n": ((.01, .25), .01),
    "lambda_s": ((1, 10), .1),
    "rhos_cs": ((1e6,1e7), 1e5),
}

all_priors = [
    ['Couche 1', Z_bottom, priors],
]

#On va maintenant lancer la MCMC

col.compute_mcmc(
    nb_iter = 400,
    all_priors = all_priors,
    nb_cells = 100,
    sigma2=1.0,
    verbose=False
)

bestParams = col.get_best_param()
print(bestParams)
print(bestParams[0])


Mcmc Computation : 100%|██████████| 400/400 [00:04<00:00, 94.21it/s] 
[Parametres(moinslog10IntrinK=15.193567154308063, n=0.17818704753901862, lambda_s=9.309925834479714, rhos_cs=8695844.526146745)]
Parametres(moinslog10IntrinK=15.193567154308063, n=0.17818704753901862, lambda_s=9.309925834479714, rhos_cs=8695844.526146745)


In [40]:
# On définit une fonction plus générale pour itérer sur tous les points de mesure et stocker les valeurs de perméabilité 

points = os.listdir(path)[1:]

def compute_MCMC(path, points):
    K_value={}
    k=len(points)/2
    for i in range(k):
        point_pression = points[2*i]
        point_temperature = points[2*i+1]
        dico = {}
        dico['chemin pression'] = os.path.join(path, point_pression)
        dico['chemin temperature'] = os.path.join(path, point_temperature)
        pression = read_csv(dico['chemin pression'])
        temperature = read_csv(dico['chemin temperature'])
        pression['temperature_stream'] = pression['temperature_stream'] + ZERO_CELSIUS
        temperature['Temp_profondeur_10'] = temperature['Temp_profondeur_10'] + ZERO_CELSIUS
        temperature['Temp_profondeur_20'] = temperature['Temp_profondeur_20'] + ZERO_CELSIUS
        temperature['Temp_profondeur_30'] = temperature['Temp_profondeur_30'] + ZERO_CELSIUS
        temperature['Temp_profondeur_40'] = temperature['Temp_profondeur_40'] + ZERO_CELSIUS
        convertDates(pression)
        convertDates(temperature)
        dico['pression'] = pression
        dico['temperature'] = temperature
        dico['temperature'].columns = ['dates', 'T_10', 'T_20', 'T_30', 'T_40']
        data_pression = list(zip(dico['pression']['dates'], list(zip( dico['pression']['dH'], dico['pression']['temperature_stream']))))
        data_temperature = list(zip(dico['temperature']['dates'], dico['temperature'][['T_10', 'T_20', 'T_30', 'T_40']].to_numpy()))
        col_dict = {
            "river_bed": .4, 
            "depth_sensors": depth_sensors, #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
            "offset": .0,
            "dH_measures": data_pression,
            "T_measures": data_temperature,
            "sigma_meas_P": 0.01, #float
            "sigma_meas_T": 0.1, #float
        }
        col = Column.from_dict(col_dict)
        col.compute_mcmc(
            nb_iter = 400,
            all_priors = all_priors,
            nb_cells = 100,
            sigma2=1.0, 
            verbose=False)

13
